In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
        
    return vocab


## UNILEX

In [3]:
df1 = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')

lista = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

df_all = pd.concat(lista)
df_all.shape

(12670, 2)

In [4]:
df_all = df_all.dropna()

In [5]:
df_all.drop(df_all[df_all['Polaridade'] == 11].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == -2].index, inplace=True)
df_all.drop(df_all[df_all['Polaridade'] == 10].index, inplace=True)

df_all['Polaridade'].unique()

array([ 0.,  1., -1.])

In [6]:
df_all.head()

,Tweet,Polaridade
0,- #CaoNossoDeCadaDia #Novo - Vanessa Mandotti ...,0.0
1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
2,- Cara de mal? Acho que não... Apenas um corte...,1.0
3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0


In [7]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [8]:
count_vect = CountVectorizer()
Tweet = count_vect.fit_transform(Tweet)
tfidf_transformer = TfidfTransformer()
X_train_UNILEX = tfidf_transformer.fit_transform(Tweet) # Aplicando o TF-IDF
X_train_UNILEX.shape

(12626, 25852)

## TWEETSENT

In [9]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [10]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [11]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [12]:
text_Tweetsent = df_tweet['text']
polarity_ = np.asarray(df_tweet['sentiment'])

In [13]:
X_text = count_vect.transform(text_Tweetsent)

X_test_TWEET = tfidf_transformer.transform(X_text) # Aplicando o TF-IDF
X_test_TWEET.shape

(11533, 25852)

## TASH

In [14]:
df1 = pd.read_csv("tash-pt.csv")
df1 = df1.dropna()
df1 = df1.drop(columns=['id_twitter'])


In [15]:
df1['text'] = df1['text'].apply(remove_user)
df1['text'] = df1['text'].apply(Tokenize)

In [16]:
text_TASH = df1['text']
sentiment_ = np.asarray(df1['sentiment'])

X_text_TASH = count_vect.transform(text_TASH)
X_text_TASH_ = tfidf_transformer.transform(X_text_TASH) # Aplicando o TF-IDF
X_text_TASH_.shape

(2787, 25852)

## KANSOAN

In [17]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [18]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [19]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

X_text_KANSOAN = count_vect.transform(text_KANSOAN)
X_text_KANSOAN_ = tfidf_transformer.transform(X_text_KANSOAN) # Aplicando o TF-IDF
X_text_KANSOAN_.shape

(9224, 25852)

## NB - TRAIN -> UNILEX- TEST -> TWEET

In [39]:
## TIRAR OS SPLITS , até no treino

In [20]:
clf = MultinomialNB()
clf.fit(X_train_UNILEX, polarity)
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred))

              precision    recall  f1-score   support

          -1       0.47      0.48      0.47      3388
           0       0.36      0.37      0.36      2928
           1       0.59      0.58      0.58      5217

    accuracy                           0.49     11533
   macro avg       0.47      0.47      0.47     11533
weighted avg       0.50      0.49      0.50     11533



## NB - TRAIN -> UNILEX - TEST -> TASH

In [21]:
clf = MultinomialNB()
clf.fit(X_train_UNILEX, polarity)
y_pred = clf.predict(X_text_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.44      0.60      0.51       881
           0       0.46      0.23      0.30      1018
           1       0.44      0.55      0.49       888

    accuracy                           0.45      2787
   macro avg       0.45      0.46      0.43      2787
weighted avg       0.45      0.45      0.43      2787



## NB - TRAIN -> TWEET - TEST -> KANSOAN

In [22]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

sentiment = df['sentiment'].apply(func)


In [23]:
clf = MultinomialNB()
clf.fit(X_train_UNILEX, polarity)
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred,zero_division=True))

              precision    recall  f1-score   support

        -1.0       0.95      0.70      0.80      4504
         0.0       0.00      1.00      0.00         0
         1.0       0.83      0.91      0.87      4720

    accuracy                           0.81      9224
   macro avg       0.59      0.87      0.56      9224
weighted avg       0.89      0.81      0.84      9224



## SVM - TRAIN->UNILEX - TEST-> TWEET

In [24]:
clf = SVC().fit(X_train_UNILEX, polarity) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred))

              precision    recall  f1-score   support

          -1       0.51      0.30      0.38      3388
           0       0.34      0.45      0.38      2928
           1       0.56      0.61      0.58      5217

    accuracy                           0.48     11533
   macro avg       0.47      0.45      0.45     11533
weighted avg       0.49      0.48      0.47     11533



## SVM T UNILEX T TASH

In [25]:
clf = SVC().fit(X_train_UNILEX, polarity) 
y_pred = clf.predict(X_text_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.47      0.40      0.43       881
           0       0.45      0.29      0.35      1018
           1       0.41      0.63      0.49       888

    accuracy                           0.43      2787
   macro avg       0.44      0.44      0.43      2787
weighted avg       0.44      0.43      0.42      2787



## SVM T UNILEX T KANSOAN

In [26]:
clf = SVC().fit(X_train_UNILEX, polarity) 
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       0.97      0.59      0.73      4504
         0.0       0.00      1.00      0.00         0
         1.0       0.81      0.93      0.86      4720

    accuracy                           0.76      9224
   macro avg       0.59      0.84      0.53      9224
weighted avg       0.88      0.76      0.80      9224



## RL T UNILEX T TWEET

In [27]:
clf = LogisticRegression(max_iter=1000).fit(X_train_UNILEX, polarity) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred))

              precision    recall  f1-score   support

          -1       0.51      0.35      0.41      3388
           0       0.35      0.38      0.36      2928
           1       0.56      0.65      0.60      5217

    accuracy                           0.49     11533
   macro avg       0.47      0.46      0.46     11533
weighted avg       0.49      0.49      0.49     11533



## RL T UNILEX T TASH

In [28]:
clf = LogisticRegression(max_iter=1000).fit(X_train_UNILEX, polarity) 
y_pred = clf.predict(X_text_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.47      0.45      0.46       881
           0       0.44      0.23      0.30      1018
           1       0.40      0.63      0.49       888

    accuracy                           0.43      2787
   macro avg       0.44      0.44      0.42      2787
weighted avg       0.44      0.43      0.41      2787



## RL T UNILEX T KANSOAN

In [29]:
clf = LogisticRegression(max_iter=1000).fit(X_train_UNILEX, polarity) 
y_pred = clf.predict(X_text_KANSOAN_)

print(classification_report(sentiment,y_pred, zero_division=True))

              precision    recall  f1-score   support

        -1.0       0.97      0.62      0.75      4504
         0.0       0.00      1.00      0.00         0
         1.0       0.81      0.94      0.87      4720

    accuracy                           0.79      9224
   macro avg       0.59      0.85      0.54      9224
weighted avg       0.89      0.79      0.81      9224



In [30]:
one = OneHotEncoder(sparse=False)
teste_ = one.fit_transform(np.asarray(sentiment).reshape(-1,1))
nome = pd.DataFrame(teste_)
nome['coluna'] = 0
nome.rename(columns={1: 2, 'coluna':1} , inplace=True)
Y_KANSOAN = nome[[0,1,2]].to_numpy()

## TOTALMENT.C T UNILEX T TWEET

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(3 , activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity).reshape(-1,1))
y_one_test = one.fit_transform(polarity_.reshape(-1,1))

fit = model.fit(X_train_UNILEX.todense(), y_one, epochs=5, validation_data=(X_test_TWEET.todense(), y_one_test))

Epoch 1/5
395/395 [==============================] - 5s 13ms/step - loss: 0.9965 - accuracy: 0.5069 - val_loss: 1.0656 - val_accuracy: 0.4455
Epoch 2/5
395/395 [==============================] - 4s 9ms/step - loss: 0.6585 - accuracy: 0.7437 - val_loss: 1.1540 - val_accuracy: 0.4873
Epoch 3/5
395/395 [==============================] - 4s 9ms/step - loss: 0.3511 - accuracy: 0.8841 - val_loss: 1.5558 - val_accuracy: 0.4872
Epoch 4/5
395/395 [==============================] - 4s 9ms/step - loss: 0.1961 - accuracy: 0.9427 - val_loss: 1.9610 - val_accuracy: 0.4883
Epoch 5/5
395/395 [==============================] - 4s 9ms/step - loss: 0.1294 - accuracy: 0.9640 - val_loss: 2.1728 - val_accuracy: 0.4876


In [32]:
predicted = model.predict(X_test_TWEET.todense())

print(classification_report(np.argmax(y_one_test, axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.46      0.42      0.44      3388
           1       0.36      0.33      0.34      2928
           2       0.56      0.62      0.59      5217

    accuracy                           0.49     11533
   macro avg       0.46      0.46      0.46     11533
weighted avg       0.48      0.49      0.48     11533



## TASH

In [33]:
predicted = model.predict(X_text_TASH_.todense())

print(classification_report(np.argmax(one.fit_transform(sentiment_.reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.45      0.48      0.46       881
           1       0.45      0.23      0.30      1018
           2       0.41      0.60      0.48       888

    accuracy                           0.43      2787
   macro avg       0.43      0.44      0.42      2787
weighted avg       0.44      0.43      0.41      2787



## KANSOAN

In [34]:
predicted = model.predict(X_text_KANSOAN_.todense())

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1), zero_division=True))

              precision    recall  f1-score   support

           0       0.94      0.69      0.80      4504
           1       0.00      1.00      0.00         0
           2       0.81      0.92      0.86      4720

    accuracy                           0.81      9224
   macro avg       0.58      0.87      0.55      9224
weighted avg       0.88      0.81      0.83      9224



## LSTM T UNILEX T TASH

In [35]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(df_all['Tweet'] )))

vocab = set_array(df_all['Tweet'] )

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


15000

In [36]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(sentiment_).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(df_all['Tweet'])), y_one, epochs=5, batch_size=128 ,validation_data=(np.asarray(pre_X(text_TASH)), y_one_test))

Epoch 1/5
99/99 [==============================] - 10s 102ms/step - loss: 1.0701 - accuracy: 0.4427 - val_loss: 1.2118 - val_accuracy: 0.3265
Epoch 2/5
99/99 [==============================] - 8s 82ms/step - loss: 0.8943 - accuracy: 0.6149 - val_loss: 1.2523 - val_accuracy: 0.3724
Epoch 3/5
99/99 [==============================] - 8s 83ms/step - loss: 0.6542 - accuracy: 0.7559 - val_loss: 1.3923 - val_accuracy: 0.3922
Epoch 4/5
99/99 [==============================] - 8s 84ms/step - loss: 0.4982 - accuracy: 0.8225 - val_loss: 1.5867 - val_accuracy: 0.3915
Epoch 5/5
99/99 [==============================] - 8s 84ms/step - loss: 0.3946 - accuracy: 0.8616 - val_loss: 1.7576 - val_accuracy: 0.3886


In [37]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.40      0.46      0.43       881
           1       0.39      0.24      0.29      1018
           2       0.38      0.49      0.43       888

    accuracy                           0.39      2787
   macro avg       0.39      0.40      0.38      2787
weighted avg       0.39      0.39      0.38      2787



## TWEET

In [38]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.45      0.37      0.40      3388
           1       0.32      0.40      0.35      2928
           2       0.53      0.51      0.52      5217

    accuracy                           0.44     11533
   macro avg       0.43      0.43      0.43     11533
weighted avg       0.45      0.44      0.45     11533



## KANSOAN

In [39]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1),zero_division=True))

              precision    recall  f1-score   support

           0       0.75      0.48      0.59      4504
           1       0.00      1.00      0.00         0
           2       0.69      0.61      0.65      4720

    accuracy                           0.55      9224
   macro avg       0.48      0.70      0.41      9224
weighted avg       0.72      0.55      0.62      9224



## CONV1D T UNILEX T KANSOAN

In [40]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity).reshape(-1,1))
y_one_test = Y_KANSOAN

fit = model.fit(np.asarray(pre_X(df_all['Tweet'])), y_one, epochs=5, batch_size=128, validation_data=(np.asarray(pre_X(text_KANSOAN)),Y_KANSOAN))

Epoch 1/5
99/99 [==============================] - 2s 25ms/step - loss: 1.0505 - accuracy: 0.4500 - val_loss: 1.2351 - val_accuracy: 0.1840
Epoch 2/5
99/99 [==============================] - 2s 21ms/step - loss: 0.8485 - accuracy: 0.6132 - val_loss: 0.9388 - val_accuracy: 0.5420
Epoch 3/5
99/99 [==============================] - 2s 21ms/step - loss: 0.5676 - accuracy: 0.7764 - val_loss: 0.8328 - val_accuracy: 0.6274
Epoch 4/5
99/99 [==============================] - 2s 22ms/step - loss: 0.3524 - accuracy: 0.8698 - val_loss: 0.9504 - val_accuracy: 0.5852
Epoch 5/5
99/99 [==============================] - 2s 23ms/step - loss: 0.2392 - accuracy: 0.9136 - val_loss: 0.9331 - val_accuracy: 0.6353


In [41]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1),zero_division=True))

              precision    recall  f1-score   support

           0       0.73      0.68      0.71      4504
           1       0.00      1.00      0.00         0
           2       0.77      0.59      0.67      4720

    accuracy                           0.64      9224
   macro avg       0.50      0.76      0.46      9224
weighted avg       0.75      0.64      0.69      9224



## TWEET

In [42]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.42      0.44      0.43      3388
           1       0.32      0.29      0.31      2928
           2       0.55      0.55      0.55      5217

    accuracy                           0.45     11533
   macro avg       0.43      0.43      0.43     11533
weighted avg       0.45      0.45      0.45     11533



## TASH

In [43]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1)))

              precision    recall  f1-score   support

           0       0.38      0.54      0.45       881
           1       0.40      0.20      0.26      1018
           2       0.38      0.44      0.41       888

    accuracy                           0.38      2787
   macro avg       0.38      0.39      0.37      2787
weighted avg       0.39      0.38      0.37      2787



## BDR T UNILEX T TWEET

In [44]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
    
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

y_one = one.fit_transform(np.asarray(polarity).reshape(-1,1))
y_one_test = one.fit_transform(np.asarray(polarity_).reshape(-1,1))

fit = model.fit(np.asarray(pre_X(df_all['Tweet'])), y_one, epochs=5, batch_size=128 ,validation_data=(np.asarray(pre_X(text_Tweetsent)), y_one_test))

Epoch 1/5
99/99 [==============================] - 14s 145ms/step - loss: 1.0225 - accuracy: 0.4907 - val_loss: 1.0687 - val_accuracy: 0.4199
Epoch 2/5
99/99 [==============================] - 13s 133ms/step - loss: 0.7413 - accuracy: 0.6909 - val_loss: 1.1083 - val_accuracy: 0.4377
Epoch 3/5
99/99 [==============================] - 13s 133ms/step - loss: 0.4830 - accuracy: 0.8131 - val_loss: 1.2714 - val_accuracy: 0.4401
Epoch 4/5
99/99 [==============================] - 14s 139ms/step - loss: 0.3347 - accuracy: 0.8739 - val_loss: 1.5498 - val_accuracy: 0.4307
Epoch 5/5
99/99 [==============================] - 14s 145ms/step - loss: 0.2584 - accuracy: 0.9007 - val_loss: 1.6781 - val_accuracy: 0.4393


In [45]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(polarity_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1),zero_division=True))

              precision    recall  f1-score   support

           0       0.43      0.41      0.42      3388
           1       0.31      0.37      0.34      2928
           2       0.54      0.49      0.52      5217

    accuracy                           0.44     11533
   macro avg       0.43      0.43      0.43     11533
weighted avg       0.45      0.44      0.44     11533



## TASH

In [46]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

print(classification_report(np.argmax(one.fit_transform(np.asarray(sentiment_).reshape(-1,1)), axis=1), np.argmax(predicted, axis=1),zero_division=True))

              precision    recall  f1-score   support

           0       0.39      0.52      0.45       881
           1       0.41      0.21      0.27      1018
           2       0.39      0.49      0.43       888

    accuracy                           0.40      2787
   macro avg       0.40      0.40      0.39      2787
weighted avg       0.40      0.40      0.38      2787



## KANSOAN

In [47]:
predicted = model.predict(np.asarray(pre_X(text_KANSOAN)))

print(classification_report(np.argmax(Y_KANSOAN, axis=1), np.argmax(predicted, axis=1),zero_division=True))

              precision    recall  f1-score   support

           0       0.72      0.64      0.68      4504
           1       0.00      1.00      0.00         0
           2       0.75      0.55      0.63      4720

    accuracy                           0.59      9224
   macro avg       0.49      0.73      0.44      9224
weighted avg       0.74      0.59      0.66      9224

